# Laboratoire 7
## Lidar, ICP et mapping

Ce laboratoire consiste à vous familiariser avec l'utilisation et le comportement de l'algorithme *Iterative Closest Point*. L'idée est d'aligner des scans lasers consécutifs de façon à construire une carte 2D de l'environnement. Notez qu'il ne s'agit pas ici de faire du SLAM, mais bien de simplement aligner des scans consécutifs sans faire de vérification ou ajustement par la suite.

![Le RPlidar A1](img/rplidar.jpeg)

Le scanner RPlidar A1 possède un champ de vision horizontal de 360 degrés et une portée de 0.02 à 12 m. Placez les obstacles à l'intérieur de cette zone pendant la création de votre carte!

### Partie 1 - Afficher un scan

D'abord la poutine préliminaire habituelle.

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib nbagg

import time
import numpy as np
import matplotlib.pyplot as plt

from robmob.icp import icp
from robmob.icp_orig import icp as icp_orig
from robmob.robot import Robot
from robmob.sensors import RPlidarSensor, FullOdomSensor

In [2]:
robot = Robot('localhost')
robot.connect()

In [3]:
rplidar = RPlidarSensor()
robot.add_sensor(rplidar)

La fonction `hokuyo_to_3d_homogeneous_matrix` accepte un scan du Hokuyo et le transforme en une matrice Nx4 de points 3D (en coordonnées homogènes). Elle fait simplement la conversion de coordonnées polaires à cartésiennes. Comme le Hokuyo est un scanneur 2D, la fonction fixe simplement la coordonnée en $z$ comme étant égale à 0.

In [6]:
def rplidar_to_3d_homogeneous_matrix(rplidar_data):
    ranges = np.array(rplidar_data['ranges'])
    thetas = np.arange(rplidar_data['angle_min'], rplidar_data['angle_max']+rplidar_data['angle_increment'], rplidar_data['angle_increment'])
    xs = ranges * np.cos(thetas)
    ys = ranges * np.sin(thetas)
    
    pts = np.vstack((xs, ys, np.zeros(xs.shape), np.ones(xs.shape))).T
    
    # Remove the points that are at the origin.
    pts = pts[np.nonzero(np.logical_or(pts[:,0] != 0., pts[:,1] != 0.))]
    
    return pts

In [7]:
pointcloud = rplidar_to_3d_homogeneous_matrix(rplidar.peek_data())
pointcloud

array([[ 1.47229552e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00],
       [ 1.46741729e+00,  2.56852461e-02,  0.00000000e+00,
         1.00000000e+00],
       [ 1.45485859e+00,  5.09464536e-02,  0.00000000e+00,
         1.00000000e+00],
       ...,
       [ 1.50707854e+00, -5.27678455e-02,  0.00000000e+00,
         1.00000000e+00],
       [ 1.49145098e+00, -2.60987528e-02,  0.00000000e+00,
         1.00000000e+00],
       [ 1.47954965e+00,  7.11255623e-06,  0.00000000e+00,
         1.00000000e+00]])

La fonction `plot_point_clouds` accepte *une liste* de ces matrices/nuages de points et les trace sur un graphique. Le graphique est dans le repère de coordonnées du lidar, ce qui veux dire que les $x$ positifs pointent vers l'arrière du robot.

In [8]:
def plot_point_clouds(pcls):
    STYLES = ['r+', 'bx', 'g*', 'md', 'ys']
    
    for i, pcl in enumerate(pcls):
        plt.plot(pcl[:,0], pcl[:,1], STYLES[i], label='Nuage {}'.format(i))
        plt.plot([0.0], [0.0], 'ko')
        
    plt.axis('equal')
    plt.legend()
    plt.show()

In [9]:
plot_point_clouds([pointcloud])

<IPython.core.display.Javascript object>

### Utiliser ICP

Nous avons une implémentation (un peu primitive) de l'algorithme ICP dans la librairie `robmob`. Pour l'utiliser, il faut appeler la fonction `icp`. Elle prend deux nuages de points en arguments et retourne une transformation homogène `H` qui exprime la position du premier nuage de point dans le repère de coordonnées du second. 

`icp` retourne aussi `err`, un vecteur qui donne la distance entre chaque paire de points associés. Ce vecteur est utile pour savoir si l'algorithme `icp` a bien convergé ou bien si le résultat laisse à désirer, avant même d'inspecter le résultat.

Pour tester l'algorithme, capturez un scan, déplacez le robot et capturez un autre scan.

> **PROTIP** Utilisez le fichier [Controller.ipynb](Controller.ipynb) pour déplacer le robot.

In [ ]:
pcl1 = rplidar_to_3d_homogeneous_matrix(rplidar.peek_data())

In [ ]:
pcl2 = rplidar_to_3d_homogeneous_matrix(rplidar.peek_data())

In [ ]:
H, err = icp(pcl1, pcl2)

In [ ]:
H

> **Attention** Au cours de ce laboratoire, nous travaillerons dans le référentiel du lidar. Comme le lidar pointe vers l'arrière du robot, c'est comme si le référentiel était inversé. Cela veut dire qu'une translation positive dans $H$ indique que le lidar a *avancé*. Par conéquent, une translation positive indique en fait que le robot a *reculé*.

Une bonne métrique pour évaluer la qualité d'un calage de nuages de points ICP est la distance moyenne entre les points couplés. Plus elle est basse,  plus les nuages de points sont compatibles entre eux. En général, avec une distance moyenne en dessous de 10 cm, vous pouvez espérer avoir un bon calage. Sinon ICP a probablement convergé dans un minimum local.

In [ ]:
print('La distance moyenne entre les points associés est de {} m'.format(np.average(err)))

Avec la matrice H en main, il est possible de déplacer le nuage `pcl1` de sorte qu'il soit dans le repère de coordonnées de `pcl2`. Complétez la fonction `pcl_with_transformation_applied`. Assurez-vous de bien respecter la documentation, sinon vous aurez des problèmes pour la suite du laboratoire.

> **Indice** Normalement, on peut faire ce déplacement seulement en multipliant deux matrices. Il est possible que certaines transpositions soient nécessaires...

In [ ]:
def pcl_with_transformation_applied(H, pcl):
    '''
    Apply an homogeneous transformation to a pointcloud.
    Input
        H: A 4x4 matrix describing an homogeneous geometric transformation.
        pcl: A Nx4 matrix containing the points to be displaced.
    Output
        transformed_pcl: A Nx4 matrix contaning the displaced points.
    '''
    return # À compléter.

L'appel suivant va tracer les deux nuages de points originaux, ainsi que le nuage de points déplacé.

> **PROTIP** Les graphiques de ce TP sont difficiles à lire. Souvenez-vous, l'objectif est de partir de `pcl1` et de venir le coller vers `pcl2`. Par exemple, dans le cas présent, Nuage 0 est le nuage d'origine et Nuage 2 sa version déplacée. Donc Nuage 2 devrait être identique à Nuage 0, mais collé sur Nuage 1.

In [ ]:
plot_point_clouds([pcl1, pcl2, pcl_with_transformation_applied(H, pcl1)])

> **Question** Est-ce que l'algorithme ICP est meilleur pour détecter les rotations ou les translations? Testez l'algorithme avec différentes rotations entre les deux scans, puis refaites l'expérience avec différentes translations entre les deux scans.

> **Question** Selon vos résultats à la question précédente, est-ce que l'ICP est capable de bien détecter de grandes transformations géométriques?

### Estimé initial

L'ICP est efficace pour trouver de petites transformations. Si on essaie de couvrir de trop grandes distances d'un scan à l'autre, l'ICP risque de tomber dans un minimum local. Pour cette raison c'est une bonne pratique de faire un estimé initial de la transformation $H$ que l'on cherche avant de lancer l'ICP. On pourrait se servir de l'odométrie, par exemple. La fonction suivante applique un estimé initial à un des nuages de points avant d'appeler ICP.

In [ ]:
def icp_with_initial_estimate(pcl1, pcl2, estimate):
    '''
    Do ICP with an initial estimate.
    Input
        pcl1, pcl2: The point clouds to be matched. Nx4 matrices.
        estimate: A 4x4 homogeneous transformation that represents our estimate of the robot displacement.
    Output
        H: A 4x4 homogeneous transformation that represents the ICP-adjusted displacement between the two PCLs.
        err: A vector containing the distance between the matched points.
    '''
    pcl1_with_estimation = pcl_with_transformation_applied(estimate, pcl1)
    H, err = icp(pcl1_with_estimation, pcl2)
    transformation_to_return = H.dot(estimate)
    
    return transformation_to_return, err

In [ ]:
pcl1 = rplidar_to_3d_homogeneous_matrix(rplidar.peek_data())

Avant de capturer le prochain scan, déplacez le robot, en notant le déplacement effectué.

In [ ]:
pcl2 = rplidar_to_3d_homogeneous_matrix(rplidar.peek_data())

In [ ]:
dx = 0.0 # Déplacement en x
dy = 0.0 # Déplacement en y
rotation = -1 # En radians
estimate = np.array([[np.cos(rotation), -np.sin(rotation), 0.0, dx],
                     [np.sin(rotation), np.cos(rotation), 0.0, dy],
                     [0.0, 0.0, 1.0, 0.0],
                     [0.0, 0.0, 0.0, 1.0]])

H, err = icp_with_initial_estimate(pcl1, pcl2, estimate)
print(H)
print('La distance moyenne entre les points associés est de {} m'.format(np.average(err)))

Une fois de plus, ce graphique est difficile à lire. L'objectif est de coller Nuage 0 sur Nuage 1. Nuage 2 est une version de Nuage 1 qui est déplacé selon notre estimé initial. Nuage 2 n'a donc jamais passé par l'ICP. Nuage 3 est une version de nuage 0 qui a été déplacée selon le résultat de l'ICP.

In [ ]:
pcl1_with_estimate = pcl_with_transformation_applied(estimate, pcl1)
pcl1_after_icp = pcl_with_transformation_applied(H, pcl1)

plot_point_clouds([pcl1, pcl2, pcl1_with_estimate, pcl1_after_icp])

> **Question** Est-ce que d'ajouter un estimé initial a aidé l'algorithme ICP à mieux converger dans le cas présent? Est-ce que l'ICP est capable de détecter correctement de plus grandes transformations géométriques avec un estimé initial?

> **Question** Est-ce que l'ICP est capable de se rétablir si vous sabotez l'estimé initial? À quelle gradeur de sabotage est-ce que l'ICP est capable de résister?

### Estimé initial grâce à l'odométrie

Il serait plus astucieux de se servir de l'odométrie pour faire un estimé initial, plutôt que de demander à un humain de le faire. La fonction `homogeneous_transformation_of_two_poses` trouve la matrice 4x4 de transformation homogène qui passe d'une position à l'autre.

In [14]:
def homogeneous_transformation_of_displacement(dx, dy, dtheta):
    return np.array([[np.cos(dtheta), -1*np.sin(dtheta), 0.0, dx],
                       [np.sin(dtheta), np.cos(dtheta), 0.0, dy],
                       [0.0, 0.0, 1.0, 0.0],
                       [0.0, 0.0, 0.0, 1.0]])

def displacement_of_two_poses(pose1, pose2):
    dx, dy, dt = [a - b for a, b in zip(pose1, pose2)]
    
    H1 = homogeneous_transformation_of_displacement(0.0, 0.0, pose1[2])
    v = np.array([[dx], [dy], [0.0], [1.0]])
    
    inv_H1 = np.linalg.inv(H1)
    rotated_v = H1.dot(v)
    
    return homogeneous_transformation_of_displacement(rotated_v[0,0], rotated_v[1,0], dt)

Le bloc suivant nous abonne aux données d'odométrie. Ensuite on peut collecter l'estimé d'odométrie en même temps que l'on connecte des scans lidar.

In [15]:
odom_sensor = FullOdomSensor()
robot.add_sensor(odom_sensor)

In [16]:
pose1 = odom_sensor.peek_data()
pcl1 = rplidar_to_3d_homogeneous_matrix(rplidar.peek_data())

In [17]:
pose2 = odom_sensor.peek_data()
pcl2 = rplidar_to_3d_homogeneous_matrix(rplidar.peek_data())

In [18]:
odom_estimate = displacement_of_two_poses(pose1, pose2)
odom_estimate

array([[ 9.99999970e-01, -2.44345786e-04,  0.00000000e+00,
        -9.38319064e-05],
       [ 2.44345786e-04,  9.99999970e-01,  0.00000000e+00,
        -2.24476170e-05],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

L'estimé d'odométrie obtenu à la case précédente est dans le référentiel du robot. On doit l'inverser pour l'avoir dans le référentiel du lidar.

In [ ]:
odom_estimate_lidar_frame = np.linalg.inv(odom_estimate)

In [ ]:
# ICP sans estimé d'odométrie.
H_without_estimate, err = icp(pcl1, pcl2)
H_without_estimate

In [ ]:
# ICP avec estimé d'odométrie.
H, err = icp_with_initial_estimate(pcl1, pcl2, odom_estimate_lidar_frame)
H

Le graphique suivant est encore plus difficile à lire, donc allez-y pas à pas. Nuage 0 est le nuage original, et Nuage 1 notre objectif. Nuage 2 est une version de nuage 0 déplacée selon notre estimé d'odométrie. Nuage 3 est une version de Nuage 0 déplacée selon l'icp avec estimé initial. Nuage 4 est une version de nuage 0 déplacé selon l'icp sans estimé initial.

In [ ]:
pcl1_with_estimate = pcl_with_transformation_applied(odom_estimate_lidar_frame, pcl1)
pcl1_after_icp = pcl_with_transformation_applied(H, pcl1)
pcl1_after_lesser_icp = pcl_with_transformation_applied(H_without_estimate, pcl1)

plot_point_clouds([pcl1, pcl2, pcl1_with_estimate, pcl1_after_icp, pcl1_after_lesser_icp])

### Construction d'une carte

Pour construire une carte de l'environnement avec le lidar, il faut intégrer tous les outils utilisés jusqu'à présent. La boucle suivante additionne les transformations successives détectées par l'ICP. On peut donc suivre le déplacement du robot dans l'environnement. 

Malheureusement l'algorithme d'ICP dont on dispose ne permet pas de faire l'ICP d'un scan lidar contre toute la carte construite. Il faut donc additionner les transformations du premier scan vers le deuxième, du deuxième vers le troisième, etc. La conséquence de cela est qu'un ICP de mauvaise qualité va affecter toutes les positions estimées successives. C'est un peu comme faire de la navigation à l'aveugle avec l'odométrie.

In [ ]:
# Le premier scan va servir de base à notre carte. On va y ajouter les scans suivants.
built_map = rplidar_to_3d_homogeneous_matrix(rplidar.peek_data())
previous_reading = built_map.copy()

sum_of_H = np.eye(4)
lidar_positions = np.array([[0.0, 0.0, 0.0, 1.0]]) # On considère que la position initiale du lidar est l'origine.
clouds = []
clouds.append(previous_reading)

for i in range(30):
    # Prendre une lecture du lidar.
    reading = rplidar_to_3d_homogeneous_matrix(rplidar.peek_data())
    clouds.append(reading)
    
    # Exécuter ICP
    H, err = icp(reading, previous_reading)
    print('La translation détectée par l\'ICP est de {:0.4f} m'.format(np.linalg.norm(H[0:1, 3])))
    
    # Ajouter la transformation trouvée à notre somme de transformations.
    sum_of_H = H.dot(sum_of_H)
    
    # Mettre à jour la position du robot.
    previous_position = lidar_positions[-1,:]
    new_position = sum_of_H.dot(np.array([0.0, 0.0, 0.0, 1.0]).T).T
    lidar_positions = np.vstack((lidar_positions, new_position))
    
    # Ajouter le scan déplacé à la carte
    displaced_reading = pcl_with_transformation_applied(sum_of_H, reading)
    built_map = np.concatenate((built_map, displaced_reading), axis=0)
    
    previous_reading = reading
    
    time.sleep(0.15)

In [ ]:
plot_point_clouds([built_map, lidar_positions])

> **Question** Pourrait-on utiliser le lidar seul pour construire la carte d'un environnement?

> **Exercice optionnel** Modifiez le code de construction de carte pour qu'il utilise l'estimé d'odométrie avant de faire l'ICP.